# 1  Data Preparation （Copy from Natasha)

In [107]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
pd.set_option('display.width',None)
#pd.set_option('display.max_colwidth',-1)

In [108]:
openpl = pd.read_csv('/Users/zhaoxuanjin/Desktop/ML/Project/openpowerlifting.csv')
openpl.head(5)

<ipython-input-108-a8d874e91391>:1: DtypeWarning: Columns (35) have mixed types. Specify dtype option on import or set low_memory=False.
  openpl = pd.read_csv('/Users/zhaoxuanjin/Desktop/ML/Project/openpowerlifting.csv')


,Name,Sex,Event,Equipment,Age,AgeClass,Division,BodyweightKg,WeightClassKg,Squat1Kg,Squat2Kg,Squat3Kg,Squat4Kg,Best3SquatKg,Bench1Kg,Bench2Kg,Bench3Kg,Bench4Kg,Best3BenchKg,Deadlift1Kg,Deadlift2Kg,Deadlift3Kg,Deadlift4Kg,Best3DeadliftKg,TotalKg,Place,Wilks,McCulloch,Glossbrenner,IPFPoints,Tested,Country,Federation,Date,MeetCountry,MeetState,MeetName
0,Abbie Murphy,F,SBD,Wraps,29.0,24-34,F-OR,59.8,60,80.0,92.5,105.0,NaN,105.0,45.0,50.0,55.0,NaN,55.0,110.0,120.0,130.0,NaN,130.0,290.0,4,324.16,324.16,286.42,511.15,NaN,NaN,GPC-AUS,2018-10-27,Australia,VIC,Melbourne Cup
1,Abbie Tuong,F,SBD,Wraps,29.0,24-34,F-OR,58.5,60,100.0,110.0,120.0,NaN,120.0,55.0,62.5,67.5,NaN,67.5,130.0,140.0,145.0,NaN,145.0,332.5,2,378.07,378.07,334.16,595.65,NaN,NaN,GPC-AUS,2018-10-27,Australia,VIC,Melbourne Cup
2,Ainslee Hooper,F,B,Raw,40.0,40-44,F-OR,55.4,56,NaN,NaN,NaN,NaN,NaN,27.5,32.5,-35.0,NaN,32.5,NaN,NaN,NaN,NaN,NaN,32.5,1,38.56,38.56,34.12,313.97,NaN,NaN,GPC-AUS,2018-10-27,Australia,VIC,Melbourne Cup
3,Amy Moldenhauer,F,SBD,Wraps,23.0,20-23,F-OR,60.0,60,-105.0,-105.0,105.0,NaN,105.0,67.5,72.5,-75.0,NaN,72.5,132.5,-140.0,-140.0,NaN,132.5,310.0,3,345.61,345.61,305.37,547.04,NaN,NaN,GPC-AUS,2018-10-27,Australia,VIC,Melbourne Cup
4,Andrea Rowan,F,SBD,Wraps,45.0,45-49,F-OR,104.0,110,120.0,130.0,140.0,NaN,140.0,70.0,75.0,80.0,NaN,80.0,150.0,160.0,170.0,NaN,170.0,390.0,3,321.25,338.91,274.56,550.08,NaN,NaN,GPC-AUS,2018-10-27,Australia,VIC,Melbourne Cup


In [109]:
openpl.drop(['Squat4Kg','Bench4Kg','Deadlift4Kg'],axis=1,inplace=True)
openpl.drop_duplicates(inplace=True)

In [110]:
#The shape of the dataset
openpl.shape

(1420270, 34)

In [111]:
def strapswraps(x):
    if x == 'Straps':
        return 'Wraps'
    return x

In [112]:
openpl['Equipment'] = openpl['Equipment'].apply(strapswraps)
print(openpl['Equipment'].value_counts())

Equipment
Single-ply    785638
Raw           466212
Wraps         103567
Multi-ply      64853
Name: count, dtype: int64


# 2. K-Nearest Neighbor

                                                                                       
# Predict: Sex, Age, BodyweightKg, Equipment, Best3SquatKg -> TotalKg 


In [113]:
# Drop Null of 5 columns

openpl.dropna(subset=['Sex','Age','BodyweightKg', 'Equipment','TotalKg','Best3SquatKg'], inplace=True)
openpl.shape

(469960, 34)

In [114]:
# Convert TotalKg 
# ( F ) TotalKg > ( F ) avg(TotalKg) -> 1 else 0  
# ( M ) TotalKg > ( M ) avg(TotalKg) -> 1 else 0

# 1 means 'Elite lifter'   0 means 'Professional lifter'

mean_valueF = openpl[openpl['Sex'] == 'F']['TotalKg'].mean() #266.87 -- F
mean_valueM = openpl[openpl['Sex'] == 'M']['TotalKg'].mean() #440.95 -- M

def tot_label(sex,score):
    if sex == 'F':
        if score <= mean_valueF:
            return 0
        else:
            return 1
    elif sex == 'M':
        if score <= mean_valueM:
            return 0
        else:
            return 1
        
openpl['Tot_label'] = openpl.apply(lambda row: tot_label(row['Sex'], row['TotalKg']), axis=1)

In [115]:
# Convert Sex to 0(F) or 1(M)
def sex_label(x):
    if(x == "F"):
        return 0
    else:
        return 1
    
openpl['Sex_label'] = pd.DataFrame(openpl.Sex.apply(lambda x : sex_label(x)))

In [116]:
# Convert Equipment to 0 or 1
equip_list = ['Single-ply','Raw','Wraps','Multi-ply']

def eq_s(x):
    if(x == 'Single-ply'):
        return 1
    else:
        return 0

def eq_r(x):
    if(x == 'Raw'):
        return 1
    else:
        return 0
def eq_w(x):
    if(x == 'Wraps'):
        return 1
    else:
        return 0

def eq_m(x):
    if(x == 'Multi-ply'):
        return 1
    else:
        return 0
    
    
openpl['Eq_s'] = pd.DataFrame(openpl.Equipment.apply(lambda x : eq_s(x)))
openpl['Eq_r'] = pd.DataFrame(openpl.Equipment.apply(lambda x : eq_r(x)))
openpl['Eq_w'] = pd.DataFrame(openpl.Equipment.apply(lambda x : eq_w(x)))
openpl['Eq_m'] = pd.DataFrame(openpl.Equipment.apply(lambda x : eq_m(x)))

In [117]:
# Scale Age
max_age = openpl['Age'].max() 
min_age = openpl['Age'].min()

openpl['Age_scaled'] = (openpl['Age'] - min_age) / (max_age - min_age)

In [118]:
# Scale BodyweightKg
max_body = openpl['BodyweightKg'].max() 
min_body = openpl['BodyweightKg'].min() 
openpl['Bodyweight_scaled'] = (openpl['BodyweightKg']-min_body)/(max_body-min_body)

In [119]:
# New dataframe
openpl[['Sex_label','Age_scaled','Bodyweight_scaled','Eq_s','Eq_r','Eq_w','Eq_m','Tot_label','Best3SquatKg']].head()

,Sex_label,Age_scaled,Bodyweight_scaled,Eq_s,Eq_r,Eq_w,Eq_m,Tot_label,Best3SquatKg
0,0,0.303665,0.187397,0,0,1,0,0,105.0
1,0,0.303665,0.181612,0,0,1,0,1,120.0
3,0,0.240838,0.188287,0,0,1,0,0,105.0
4,0,0.471204,0.384095,0,0,1,0,1,140.0
5,0,0.387435,0.250590,0,0,1,0,1,142.5


In [120]:
X = openpl[['Sex_label','Age_scaled','Bodyweight_scaled','Eq_s','Eq_r','Eq_w','Eq_m','Best3SquatKg']]
Y = openpl['Tot_label']

# Data Splitting

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.25, random_state=42)

In [121]:
# Train a k-NN model 
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 30)

knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=30)

In [122]:
knn.predict(X_test)
print('Train score:',knn.score(X_train, y_train))
print('Test score:',knn.score(X_test, y_test))

Train score: 0.9315175759639118
Test score: 0.9259851902289556
